# Setup

In [3]:
# install ProfileReport
#!pip install ydata-profiling

In [4]:
# import common libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import specific components from scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# enhanced stats functions
from scipy import stats

# for ease of data profiling
from ydata_profiling import ProfileReport

C:\Users\bethe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# print environment setup details
print(f"pandas version  : {pd.__version__}")  # 2.0.2
print(f"numpy version   : {np.__version__}")  # 1.24.3
print(f"seaborn version : {sns.__version__}") # 0.12.2

pandas version  : 2.0.2
numpy version   : 1.24.3
seaborn version : 0.12.2


Here, we also set the seed for numpy's random number generator such that our results are fully reproducible. This is because the other libraries (e.g. scikit-learn) use this random number generator, so if we set the seed we will always generate the same random numbers in the same sequence.

Thus, whenever we run the notebook from top-to-bottom, we will end up with the *exact* same results! 

In [6]:
SEED = 123
np.random.seed(SEED)

# Data Loading
To load in the data for this project, read in `insurance.csv` into a variable called `df` as a pandas DataFrame. The first two rows of the DataFrame should look like this:

|       | age | sex     | bmi    | children | smoker | region    | charges     |
| ----- | --- | ------- | ------ | -------- | ------ | --------- | ----------- |
| **0** | 19  | female  | 27.900 | 0        | yes    | southwest | 16884.92400 |
| **1** | 18  | male    | 33.770 | 1        | no     | southeast | 1725.55230  |

In [12]:
df = pd.read_csv("C:/Users/bethe/Downloads/insurance.csv")

In [13]:
# make sure data was read in properly and matches above table
df.head(2)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


# EDA
Now it is time for you to practice your EDA skills! Take a look at the Quest 2 Quiz, and answer the first 5 questions on the results of your EDA on the dataset.

**Data dictionary**

| column | data definition | 
| --- | --- | 
| age | age of insured person | 
| sex | sex of insured person, either male or female | 
| bmi | body mass index of insured person |
| children | number of dependents covered by health insurance |
| smoker | does the insured person smoke? |
| region | the insured person's residential area within the US |
| charges | medical costs billed by health insurance; target variable |

For additional information, check the dataset's [information on Kaggle](https://www.kaggle.com/datasets/mirichoi0218/insurance).

In [14]:
# optional: use pandas-profiling to do quick first pass at EDA
profile = ProfileReport(df, title="Insurance Dataset Profiling Report")
# create html file to view report
profile.to_file("insurance_dataset_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 112.25it/s]


In [ ]:
# conduct any other EDA that you need to in order to get a good feel for the data
df.describe()

With the EDA you have conducted, answer the following questions from the quiz. Note that these questions do not cover everything you should be looking for when doing EDA, they are just to give you an idea of what EDA would look like on such a dataset.
## Question 1
What is the median age of the individuals in our dataset?

In [ ]:
# calculate median age


## Question 2
What is the index of the person who has the highest BMI?

In [ ]:
# find index of person with highest BMI


## Question 3
Which two independent numeric variables (excluding categorical and dummy variables) are most highly correlated with each other?

In [ ]:
# plot correlation heatmap


## Question 4
What is the distribution of the `charges` variable in our dataset?

In [ ]:
# plot distribution of charges


That's strange, that doesn't look like our data is normally distributed. While you can theoretically still apply linear regression to a distribution which isn't normally distributed, it's generally best practice for your data to be normal. Looks like we may need to transform our data in some way in order to get it closer to a normal distribution.

Create a new column in df, `log_charges`, by taking the log of the existing `charges` column. Then, drop the `charges` column. Your first two rows should now look like:

|       | age | sex     | bmi    | children | smoker | region    | log_charges |
| ----- | --- | ------- | ------ | -------- | ------ | --------- | ----------- |
| **0** | 19  | female  | 27.900 | 0        | yes    | southwest | 9.734176    |
| **1** | 18  | male    | 33.770 | 1        | no     | southeast | 7.453302    |

In [ ]:
# take log of charges, then drop old column
df["log_charges"] = 

In [ ]:
# check if your table matches the one above
df.head(2)

Great, now plot the new distribution to see whether it's a normal distribution.

In [ ]:
# plot distribution for log_charges

Note that even after log transforming our data, it is still not normally distributed. But it is now closer to a normal distribution than the data originally was, so we can continue on with our analysis using this new variable `log_charges` variable as our target variable instead!

### Optional exploration
Are there other ways of determining whether the data is normally distributed or not, besides looking at the above plot? Hint: they're in the stats package we already imported from scipy!

## Question 5
There are two rows which are identical to each other; what are their indexes?

In [ ]:
# find indexes of duplicated rows

# Data Wrangling
Now that we have an idea of what our data looks like, we need to start preparing it for modelling.

In this case, we don't have any missing data, so we don't need to do anything about that. We have one duplicated row, but that seems to be legitimate data, so we will keep it in. 

We have 3 categorical variables - sex, smoker, and region. Get dummy variables using pandas, ensuring to use the `drop_first=True` argument to mitigate possible multicollinearity issues. As a result, you should only have one dummy variable for binary values such as `sex` or `smoker`. 

In [ ]:
# get dummies for sex, smoker, and region


Now verify that we still don't have major multicollinearity issues with a heatmap. What we're looking for is that most of the correlations between independent variables should still be relatively low. There's no single cut-off value (much of this is as much an art as it is a science), but we'll say for our purposes here that we'll consider anything between -0.5 and 0.5 to be low.

Note that if a variable is strongly correlated with our dependent variable `log_charges`, that's okay. If anything, that's probably a good thing!

In [ ]:
# correlation plot heatmap


## Question 6
After getting dummy variables from our categorical variables, how many independent variables do we have?

In [ ]:
# get number of independent variables


Now separate our independent variables into a variable called `X`, and our target variable `log_charges` into a variable called `y`.

In [ ]:
# split up our variables
X = 
y = 

In [ ]:
# the resulting shapes of X and y should be (1338, 8) and (1338,) respectively
print(X.shape)
print(y.shape)

Now we need to split up our data into training and test data. Using scikit-learn's `train_test_split` function, using a `test_size` of 0.3 (i.e. 30% of data in test set), and **ensure that the random state is set to our seed from above**.

Documentation for `train_test_split()` can be found here: https://scikit-learn.org/1.1/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# do the train test split
X_train, X_test, y_train, y_test = 

### Question 7
What are the indexes for the first five data points in the training dataset after train_test_split?

# Modelling and Evaluation
Now we can make our model! Instantiate a LinearRegression model in scikit-learn, then fit the training data on it.

Documentation for `LinearRegression()` can be found here: https://scikit-learn.org/1.1/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# instantiate linear regression model and fit the training data to it
lr = 

In [ ]:
# describe model
print(f"intercept: {lr.intercept_:.4f}")
for i in range(len(X_train.columns)):
    print(f"{X_train.columns[i]}: {lr.coef_[i]:.4f}")

Now we need to evaluate how our model did. Let's start by gathering some basic metrics on the training data (e.g. MAE, RMSE, R^2), then do the same on the test set.

In [ ]:
# predict training set values
predictions_train = 

In [ ]:
# evaluate model R^2 (score)
r2_train = 

In [ ]:
# evaluate model MAE
mae_train = 

In [ ]:
# evaluate model RMSE
rmse_train = 

In [ ]:
# print metrics
print("metrics for training data")
print(f"R^2 score : {r2_train:.4f}")
print(f"mae       : {mae_train:.4f}")
print(f"rmse      : {rmse_train:.4f}")

In [ ]:
# plot predictions against actual values
plt.scatter(y_train, predictions_train)
plt.title("training dataset")
plt.xlabel("actual")
plt.ylabel("predicted")
plt.gca().axline([7, 7], [11, 11], color="red")
plt.grid()

Clearly, while our model has clearly learned something about the data, there is still room for improvement.

Now we're going to evaluate our model's performance on the test set. First, we need to make our model predict the values for the test set.

In [ ]:
# make predictions on test set
predictions_test = 

In [ ]:
# get same metrics as before - R^2, MAE, RMSE
r2_test = 
mae_test = 
rmse_test = 

In [ ]:
# print metrics
print("metrics for test data")
print(f"R^2 score : {r2_test:.4f}")
print(f"mae       : {mae_test:.4f}")
print(f"rmse      : {rmse_test:.4f}")

In [ ]:
# plot predictions against actual values
plt.scatter(y_test, predictions_test)
plt.title("test dataset")
plt.xlabel("actual")
plt.ylabel("predicted")
plt.gca().axline([7, 7], [11, 11], color="red")
plt.grid()

## Question 8
Are the residuals normally distributed?

In [ ]:
# plot residuals to determine if they are normally distributed


Now, let's take a step back. We know that we took the log of `charges` in order to make it closer to a normal distribution. While this is helpful for our modelling, at the end of the day we want to predict the actual insurance charges, not the log of them.

## Question 9
What are the predicted insurance charges for index 650 (the first row in the test dataset), rounded to 2 decimal places (i.e. nearest cent)?

In [ ]:
# get predicted insurance charge for index 650
charge = 

In [ ]:
# get the predicted insurance charges
print(f"predicted insurance charges: {charge:.2f}")

# Discussion
While our model has learned some of the relationships from the data, it is still imperfect. What else could we do to improve it? Here are some possibilities:
1. We did not normalize/standardize our data, which could result in larger numeric variables (e.g. age) having a larger impact on the final result than smaller numeric variables (e.g. children). For more on normalization and standardization, check out [this resource](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/).
2. As our region dummy variables seem somewhat negatively correlated with each other, perhaps we may have issues with multicollinearity. So we could try encoding the region variable differently, separating into just 2 variables, one for north/south and the other for east/west.
3. Perhaps some of the variables interact with each other. We could try adding additional features covering such interactions (e.g. smoking & sex, region & bmi, etc.). Check out [this resource for more information on interaction effects](https://stattrek.com/multiple-regression/interaction).
4. Instead of normalizing our `charges` value by taking the log of it, we could do a Box-Cox transformation instead. Check out [this resource for more on Box-Cox transformation](https://towardsdatascience.com/box-cox-transformation-explained-51d745e34203).

Implementing any of the suggested potential improvements can be a task for you to work on